## Comparison between Adam and SGD optimizers, with different initialisations

In [1]:
from __future__ import print_function, division
import sys
sys.path.append('../src')
sys.path.append('../script')

import time
import torch
import pickle
from tqdm.notebook import tqdm
from IPython.utils import io
import torch.nn as nn
from torch.nn import Parameter
import LocalEnergyVct as le
import numpy as np
from torch.utils.data import DataLoader, random_split
from data_classes import RNASeqDataset, LocalEnergyOpt
from sklearn.metrics import r2_score
# from my_script import get_target, loss_fn, train, test
import matplotlib.pyplot as plt
plt.style.use('bmh')

In [2]:
def get_target(X):
    if len(X['features'].shape) == 2:
        X['features'] = X['features'].unsqueeze(0)
    target = (X['features'][:,0:3,9]).sum(dim=0).squeeze() / X['features'].shape[0]
    return target


# Functions without gradients in the loss

def loss_fn(energy,target):
    # batch_size = energy.shape[0]
    loss = (energy - target).pow(2).sum()  # / batch_size
    return loss


def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    model.train()
    train_loss = 0
    y_true = torch.zeros((num_batches,3))
    y_pred = torch.zeros((num_batches,3))
    for i,X in enumerate(dataloader):
        pred = model(X)
        target = get_target(X)
        y_true[i] = target.reshape(-1,)  # requires drop_last = True
        y_pred[i] = pred.reshape(-1,)
        loss = loss_fn(pred, target)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    y_true = y_true.detach().numpy()
    y_pred = y_pred.detach().numpy()
    acc = r2_score(y_true,y_pred)
    train_loss /= num_batches
    print(f'Avg loss = {train_loss:>0.4f}, batches = {num_batches}')
    return train_loss, acc


def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    y_true = torch.zeros((num_batches,3))
    y_pred = torch.zeros((num_batches,3))
    with torch.no_grad():
        for i,X in enumerate(dataloader):
            pred = model(X)
            target = get_target(X)
            y_true[i] = target.reshape(-1,) # requires drop_last = True
            y_pred[i] = pred.reshape(-1,)
            loss = loss_fn(pred, target)
            test_loss += loss.item()
    y_true = y_true.detach().numpy()
    y_pred = y_pred.detach().numpy()
    acc = r2_score(y_true,y_pred)
    test_loss /= num_batches
    print(f'Avg test_loss = {test_loss:>0.4f}, batches = {num_batches}')
    return test_loss, acc


def KL_divergence(hist1,hist2):
    # div(p,q) = sum_x p(x) * log(p(x)/q(x))
    # hist[1]: bin limits
    # hist[0]: y value
    p_x = hist1[0] * np.diff(hist1[1])
    q_x = hist2[0] * np.diff(hist2[1])
    cond = (q_x>0) & (p_x>0)
    div = np.sum( -p_x[cond] * np.log(q_x[cond]/p_x[cond]))
    return div


def plot_hist(idx_dict,energies):
    fig,ax = plt.subplots(1,3,figsize=(18,5))
    for i in idx_dict.keys():
        hist1 = ax[i].hist(energies['amber'][i], bins=30, density=True, label='Amber')
        ax[i].hist(energies['hire'][i], bins=hist1[1], alpha=0.6, density=True, label='HiRE')
        ax[i].set_xlabel(idx_dict[i]+' energy', fontsize=15)
        ax[i].set_ylabel('Prob. distribution', fontsize=15)
        ax[i].set_title(idx_dict[i]+' energy distribution', fontsize=18)
        ax[i].legend(fontsize=15)
    return 0


def compare_energies(dataset,model,plot=False):
    
    idx_dict = {
        0: 'Bonds',
        1: 'Angles',
        2: 'Torsions'
    }
    energies = {'amber': [], 'hire': []}
    stats = {'amber': [], 'hire': []}
    for i in idx_dict.keys():
        amber_en = np.array([dataset[j]['features'][i,9].item() for j in range(len(dataset))])
        hire_en = np.array([model(dataset[j]).squeeze()[i].item() for j in range(len(dataset))])
        energies['amber'].append(amber_en)
        energies['hire'].append(hire_en)
        stats['amber'].append([amber_en.mean(), amber_en.var()])
        stats['hire'].append([hire_en.mean(), hire_en.var()])
        print(idx_dict[i]+' energy computed')
    stats['amber'] = np.array(stats['amber'])
    stats['hire'] = np.array(stats['hire'])
    
    if plot:
        plot_hist(idx_dict,energies)
        
    return energies,stats


In [3]:
# Define a sort of distances between distributions of energy
# Both Kullback-Leibler divergence and total variation distance were implemented

def KL_tensor_divergence(hist1,hist2):
    # div(p,q) = sum_x p(x) * log(p(x)/q(x))
    # hist[1]: bin limits
    # hist[0]: y value
    p_x = hist1[0] * torch.diff(hist1[1])
    q_x = hist2[0] * torch.diff(hist2[1])
    cond = (q_x>0) & (p_x>0)
    div = torch.sum( -p_x[cond] * torch.log(q_x[cond]/p_x[cond]))
    return div


def TotVarDist(hist1,hist2):
    return torch.sum(torch.abs(hist1[0]-hist2[0]) * torch.diff(hist1[1])).item()


def get_amber_hist(dataset):
    n_bins = 30
    amber_hist_bins = torch.zeros((3,n_bins+1))
    amber_hist_values = torch.zeros((3,n_bins))
    for i in range(3):
        amber_en = torch.tensor([dataset[j]['features'][i,9].item() for j in range(len(dataset))])
        amber_hist_values[i], amber_hist_bins[i] = torch.histogram(amber_en, bins=n_bins, density=True)
    return (amber_hist_values, amber_hist_bins)


def KL_train(dataloader, model, loss_fn, optimizer, amber_hist):
    num_batches = len(dataloader)
    train_loss = 0
    y_pred = torch.zeros((len(dataloader.dataset),3))
    KL_score = 0
    model.train()    
    for i,X in enumerate(dataloader):
        pred = model(X)
        target = get_target(X)
        y_pred[i: i+dataloader.batch_size] = pred
        loss = loss_fn(pred, target)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= num_batches
    
    for i in range(3):
        hist1 = (amber_hist[0][i],amber_hist[1][i])
        hist2 = torch.histogram(y_pred[:,i], bins=hist1[1], density=True)
        KL_score += TotVarDist(hist1,hist2)  # KL_tensor_divergence(hist1,hist2)
        
    print(f'Avg loss = {train_loss:>0.4f}, batches = {num_batches}')
    return train_loss, KL_score


def KL_test(dataloader, model, loss_fn, amber_hist):
    num_batches = len(dataloader)
    test_loss = 0
    y_pred = torch.zeros((len(dataloader.dataset),3))
    KL_score = 0
    model.eval()
    with torch.no_grad():
        for i,X in enumerate(dataloader):
            pred = model(X)
            target = get_target(X)
            y_pred[i: i+dataloader.batch_size] = pred
            loss = loss_fn(pred, target)
            test_loss += loss.item()
    
    for i in range(3):
        hist1 = (amber_hist[0][i],amber_hist[1][i])
        hist2 = torch.histogram(y_pred[:,i], bins=hist1[1], density=True)
        KL_score += TotVarDist(hist1,hist2)  # KL_tensor_divergence(hist1,hist2)
    
    test_loss /= num_batches
    print(f'Avg test_loss = {test_loss:>0.4f}, batches = {num_batches}')
    return test_loss, KL_score

In [4]:
# CUDA for Pytorch
print(torch.cuda.is_available())
# device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device ='cpu'

True


In [ ]:
# Parameters
params = {'batch_size': 1,
          'shuffle': True,
          'drop_last': True,
          'num_workers': 0,
          'pin_memory': False}

# Datasets and Dataloaders
seq_data = RNASeqDataset(device=device)
print(f'dataset allocated on {device}')
tot_length = len(seq_data)
test_length = int(0.2 * tot_length)
train_set, test_set = random_split(seq_data, [tot_length - test_length, test_length])  #, generator=torch.Generator().manual_seed(42))
print(f'Training set: {len(train_set)} elements')
print(f'Test set: {len(test_set)} elements')
train_dataloader = DataLoader(train_set,**params)
test_dataloader = DataLoader(test_set,**params)

fixed_pars = pickle.load(open('../data/SeqCSV/fixed_pars.p', 'rb'))
opt_pars = pickle.load(open('../data/SeqCSV/pars.p', 'rb'))

# set parameters to the same order of magnitude
fixed_pars['bond_type'][:,0] /= 100
fixed_pars['angle_type'][:,0] /= 100

model = LocalEnergyOpt().to(device)
# torch.save(model.state_dict(), 'data/Results/initial_values_all1.pth')

my_loss = loss_fn  # _with_grad
my_train = train  # _with_grad
my_test = test  # _with_grad

for p in model.parameters():
    print(p)

In [ ]:
lr = 1e-4
adam_optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_loss_adam = []
test_loss_adam = []
train_acc_adam = []
test_acc_adam = []

epochs = 150
for index_epoch in range(epochs):
    print(f'epoch {index_epoch+1}/{epochs} \n-------------------------')
    t0 = time.time()
    with io.capture_output() as captured:
        train_tmp, train_acc_tmp = my_train(train_dataloader, model, my_loss, adam_optimizer)
        test_tmp, test_acc_tmp = my_test(test_dataloader, model, my_loss)
    tf = time.time()
    print(f'Avg train loss = {train_tmp:.4f}, train accuracy = {train_acc_tmp:.4f}')
    print(f'Avg test loss = {test_tmp:.4f}, test accuracy = {test_acc_tmp:.4f}' )
    print(f'time for epoch: {tf-t0:.2f} \n')
    train_loss_adam.append(train_tmp)
    test_loss_adam.append(test_tmp)
    train_acc_adam.append(train_acc_tmp)
    test_acc_adam.append(test_acc_tmp)
    
torch.save(model.state_dict(), '../results/Results_fixedLR/150_b1_e4_Aall1.pth')  # epochs_batchsize_lr_A/SGD + all1/so/~

for p in model.parameters():
    print(p.data)

In [ ]:
model2 = LocalEnergyOpt(fixed_pars,opt_pars,device,set_to_one=True).to(device)
lr = 5e-7
SGD_optimizer = torch.optim.SGD(model2.parameters(), lr=lr, momentum=0.9)  # consider adding momentum
scheduler = scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(SGD_optimizer, 'min', factor=0.5, patience=0, cooldown=1000, threshold=1e-10, verbose=True)
train_loss_SGD = []
test_loss_SGD = []
train_acc_SGD = []
test_acc_SGD = []

epochs = 150
for index_epoch in range(epochs):
    print(f'epoch {index_epoch+1}/{epochs} \n-------------------------')
    t0 = time.time()
    with io.capture_output() as captured:
        train_tmp, train_acc_tmp = my_train(train_dataloader, model2, my_loss, SGD_optimizer)
        test_tmp, test_acc_tmp = my_test(test_dataloader, model2, my_loss)
    tf = time.time()
    print(f'Avg train loss = {train_tmp:.4f}, train accuracy = {train_acc_tmp:.4f}')
    print(f'Avg test loss = {test_tmp:.4f}, test accuracy = {test_acc_tmp:.4f}' )
    print(f'time for epoch: {tf-t0:.2f} \n')
    train_loss_SGD.append(train_tmp)
    test_loss_SGD.append(test_tmp)
    train_acc_SGD.append(train_acc_tmp)
    test_acc_SGD.append(test_acc_tmp)
    
torch.save(model.state_dict(), '../results/Results_fixedLR/150_b1_5e7_SGDso.pth')  # epochs_batchsize_lr_A/SGD + all1/so/~

for p in model.parameters():
    print(p.data)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(14,5))
ax[0].semilogy(train_loss_adam, label='Adam')
ax[0].semilogy(train_loss_SGD, label='SGD')
ax[0].set_xlabel('Epochs', fontsize=15)
ax[0].set_ylabel('Loss', fontsize=15)
ax[0].legend(fontsize=15)
ax[0].set_title('Train loss', fontsize=18)

ax[1].semilogy(test_loss_adam, label='Adam')
ax[1].semilogy(test_loss_SGD, label='SGD')
ax[1].set_xlabel('Epochs', fontsize=15)
ax[1].set_ylabel('Loss', fontsize=15)
ax[1].legend(fontsize=15)
ax[1].set_title('Test loss', fontsize=18)
plt.show()

In [ ]:
plt.semilogy(train_loss_SGD)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(14,5))
ax[0].plot(train_acc_adam, label='Adam')
ax[0].plot(train_acc_SGD, label='SGD')
ax[0].axhline(y=1, color='black', linestyle='--', linewidth=0.7)
ax[0].set_xlabel('Epochs', fontsize=15)
ax[0].set_ylabel('R2 score', fontsize=15)
ax[0].set_ylim([-2.2,1.5])
ax[0].legend(fontsize=15)
ax[0].set_title('Train score, batch size = 1', fontsize=18)

ax[1].plot(test_acc_adam, label='Adam')
ax[1].plot(test_acc_SGD, label='SGD')
ax[1].axhline(y=1, color='black', linestyle='--', linewidth=0.7)
ax[1].set_xlabel('Epochs', fontsize=15)
ax[1].set_ylabel('R2 score', fontsize=15)
ax[1].set_ylim([-2.2,1.5])
ax[1].legend(fontsize=15)
ax[1].set_title('Test score, batch size = 1', fontsize=18)

# plt.savefig('Images/Score_400_b16.pdf')
plt.show()

## Adam optimizer analysis: learning rate and batch size

In [ ]:
_,stats_adam = compare_energies(seq_data,model,plot=True)

In [ ]:
_,stats_SGD = compare_energies(seq_data,model2,plot=True)

Adam results seem better in terms of matching between obtained distributions and original ones. Indeed, one issue to take into consideration is that, if the learning rate is too high (indicatively above $10^{-4}$), it leads to negative couplings, which is obviously unphysical.
Limitations to SGD are instead the learning rate, that needs to be small to avoid divergent quantities, and wrong predictions concerning the variance of distributions. Even an increase in the batch size does not modify substantially these behaviours.

In particular, the larger discrepancy is observed in the angle energy distribution

In [ ]:
print(stats_adam['amber']-stats_adam['hire'])
print(stats_SGD['amber']-stats_SGD['hire'])

In [ ]:
amber_hist = get_amber_hist(seq_data)
# print(amber_hist[0][0], amber_hist[1][0])

In [ ]:
model.load_state_dict(torch.load("../results/Results_fixedLR/initial_values_sameorder.pth"))
lr = 1e-4
adam_optimizer = torch.optim.Adam(model.parameters(), lr=lr)
my_train = KL_train
my_test = KL_test
epochs = 200

train_loss_adam = []
test_loss_adam = []
train_acc_adam = []
test_acc_adam = []

for index_epoch in range(epochs):
    print(f'epoch {index_epoch+1}/{epochs} \n-------------------------')
    t0 = time.time()
    with io.capture_output() as captured:
        train_tmp, train_acc_tmp = my_train(train_dataloader, model, my_loss, adam_optimizer, amber_hist)
        test_tmp, test_acc_tmp = my_test(test_dataloader, model, my_loss, amber_hist)
    tf = time.time()
    print(f'Avg train loss = {train_tmp:.4f}, train score = {train_acc_tmp:.4f}')
    print(f'Avg test loss = {test_tmp:.4f}, test score = {test_acc_tmp:.4f}' )
    print(f'time for epoch: {tf-t0:.2f} \n')
    train_loss_adam.append(train_tmp)
    test_loss_adam.append(test_tmp)
    train_acc_adam.append(train_acc_tmp)
    test_acc_adam.append(test_acc_tmp)

In [ ]:
plt.plot(train_acc_adam, label='Train')
plt.plot(test_acc_adam, label='Test')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.7)
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('TVD score', fontsize=15)
plt.ylim([-0.5,3])
plt.legend(fontsize=15)
plt.title('TVD score, batch size = 1', fontsize=18)

In [ ]:
for p in model.parameters():
    print(p)

In [ ]:
# try same hyperparameters, but set initial values to 1


model2 = LocalEnergyOpt(fixed_pars,opt_pars,device,set_to_one=True).to(device)
for p in model2.parameters():
    print(p)
lr = 1e-4

adam_optimizer = torch.optim.Adam(model2.parameters(), lr=lr)
my_train = KL_train
my_test = KL_test
epochs = 100

train_loss_adam2 = []
test_loss_adam2 = []
train_acc_adam2 = []
test_acc_adam2 = []

for index_epoch in range(epochs):
    print(f'epoch {index_epoch+1}/{epochs} \n-------------------------')
    t0 = time.time()
    with io.capture_output() as captured:
        train_tmp, train_acc_tmp = my_train(train_dataloader, model2, my_loss, adam_optimizer, amber_hist)
        test_tmp, test_acc_tmp = my_test(test_dataloader, model2, my_loss, amber_hist)
    tf = time.time()
    print(f'Avg train loss = {train_tmp:.4f}, train score = {train_acc_tmp:.4f}')
    print(f'Avg test loss = {test_tmp:.4f}, test score = {test_acc_tmp:.4f}' )
    print(f'time for epoch: {tf-t0:.2f} \n')
    train_loss_adam2.append(train_tmp)
    test_loss_adam2.append(test_tmp)
    train_acc_adam2.append(train_acc_tmp)
    test_acc_adam2.append(test_acc_tmp)

In [ ]:
plt.plot(train_acc_adam2, label='Train')
plt.plot(test_acc_adam2, label='Test')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.7)
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('TVD score', fontsize=15)
plt.ylim([-0.5,3])
plt.legend(fontsize=15)
plt.title('TVD score, batch size = 1', fontsize=18)

In [ ]:
for p in model2.parameters():
    print(p)

In [ ]:
_,stats_adam_so = compare_energies(seq_data,model,plot=True)
_,stats_adam_all1 = compare_energies(seq_data,model2,plot=True)

Starting with same parameters for Adam, but with all initial values for the coupling constants set to 1 leads to a similar score, but unfortunately one of the parameters becomes negative. Nevertheless, it is interesting to notice that, even if the value are different, some patterns can be found, especially in the relations between the coefficients.
For example, the first bond coupling tends to go to a lower value than the others, independently of the initialization, and similar patterns are obtained also for angle parameters, especially the first ones.

In [ ]:
pars_so = [p.data for p in model.parameters()]
pars_1 = [p.data for p in model2.parameters()]

plt.plot(pars_so[0],'o')
plt.plot(pars_1[0],'o')
plt.ylim(0,7)
plt.show()
print(pars_so[0][0])
print(pars_1[0][0])

# x and y coordinates are coupling constant and equilibrium value, respectively

plt.scatter(pars_so[1][:,0]*pars_so[0][0],pars_so[1][:,1],label='Same Order')
plt.scatter(pars_1[1][:,0]*pars_1[0][0],pars_1[1][:,1],label='All 1')
for i in range(len(pars_so[1])):
    plt.annotate(i+1, (pars_so[1][i,0]*pars_so[0][0],pars_so[1][i,1]))
    plt.annotate(i+1, (pars_1[1][i,0]*pars_1[0][0],pars_1[1][i,1]))
plt.ylim(0,4)
plt.legend()
plt.show()

plt.scatter(pars_so[2][:,0]*pars_so[0][-1],pars_so[2][:,1],label='Same Order')
plt.scatter(pars_1[2][:,0]*pars_1[0][-1],pars_1[2][:,1],label='All 1')
for i in range(len(pars_so[2])):
    plt.annotate(i+1, (pars_so[2][i,0]*pars_so[0][-1],pars_so[2][i,1]))
    plt.annotate(i+1, (pars_1[2][i,0]*pars_1[0][-1],pars_1[2][i,1]))
plt.legend()
plt.show()

plt.scatter(pars_so[3][:,0],pars_so[3][:,1],label='Same Order')
plt.scatter(pars_1[3][:,0],pars_1[3][:,1],label='All 1')
for i in range(len(pars_so[3])):
    plt.annotate(i+1, (pars_so[3][i,0],pars_so[3][i,1]))
    plt.annotate(i+1, (pars_1[3][i,0],pars_1[3][i,1]))
plt.legend()
plt.show()

In [ ]:
amber_hist = get_amber_hist(seq_data)

In [ ]:
plt.plot(train_acc_adam, label='Train')
plt.plot(test_acc_adam, label='Test')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.7)
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('TVD score', fontsize=15)
plt.ylim([-0.5,3])
plt.legend(fontsize=15)
plt.title('TVD score, batch size = 1', fontsize=18)

In [ ]:
for p in model.parameters():
    print(p)

In [ ]:
# try same hyperparameters, but set initial values to 1


model2 = LocalEnergyOpt(fixed_pars,opt_pars,device,set_to_one=True).to(device)
for p in model2.parameters():
    print(p)
lr = 1e-4

adam_optimizer = torch.optim.Adam(model2.parameters(), lr=lr)
my_train = KL_train
my_test = KL_test
epochs = 200

train_loss_adam2 = []
test_loss_adam2 = []
train_acc_adam2 = []
test_acc_adam2 = []

for index_epoch in range(epochs):
    print(f'epoch {index_epoch+1}/{epochs} \n-------------------------')
    t0 = time.time()
    with io.capture_output() as captured:
        train_tmp, train_acc_tmp = my_train(train_dataloader, model2, my_loss, adam_optimizer, amber_hist)
        test_tmp, test_acc_tmp = my_test(test_dataloader, model2, my_loss, amber_hist)
    tf = time.time()
    print(f'Avg train loss = {train_tmp:.4f}, train score = {train_acc_tmp:.4f}')
    print(f'Avg test loss = {test_tmp:.4f}, test score = {test_acc_tmp:.4f}' )
    print(f'time for epoch: {tf-t0:.2f} \n')
    train_loss_adam2.append(train_tmp)
    test_loss_adam2.append(test_tmp)
    train_acc_adam2.append(train_acc_tmp)
    test_acc_adam2.append(test_acc_tmp)

In [ ]:
plt.plot(train_acc_adam2, label='Train')
plt.plot(test_acc_adam2, label='Test')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.7)
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('TVD score', fontsize=15)
plt.ylim([-0.5,3])
plt.legend(fontsize=15)
plt.title('TVD score, batch size = 1', fontsize=18)

In [ ]:
for p in model2.parameters():
    print(p)

Starting with same parameters for Adam, but with all initial values for the coupling constants set to 1 leads to a similar score, but unfortunately one of the parameters becomes negative. Nevertheless, it is interesting to notice that, even if the value are different, some patterns can be found, especially in the relations between the coefficients.
For example, the first bond coupling tends to go to a lower value than the others, independently of the initialization, and similar patterns are obtained also for angle parameters, especially the first ones.

In [5]:
modelso = LocalEnergyOpt()
modelso.load_state_dict(torch.load('../results/Results_fixedLR/150_b1_e4_Aso.pth'))
model1 = LocalEnergyOpt()
model1.load_state_dict(torch.load('../results/Results_fixedLR/150_b1_e4_Aall1.pth'))
modelin = LocalEnergyOpt()
modelin.load_state_dict(torch.load("../results/Results_fixedLR/initial_values_sameorder.pth"))

<All keys matched successfully>

In [ ]:
all_bonds = torch.cat((modelin.bond_type.data, model1.bond_type.data, modelso.bond_type.data), dim=1)
print(all_bonds)

In [ ]:
all_angles = torch.cat((modelin.angle_type.data, model1.angle_type.data, modelso.angle_type.data), dim=1)
print(all_angles)

In [ ]:
all_tors = torch.cat((modelin.tor_type.data, model1.tor_type.data, modelso.tor_type.data), dim=1)
torch.set_printoptions(linewidth=120, sci_mode=False)
print(all_tors)